In [1]:

from __future__ import print_function
import numpy as np
from keras.layers import Input, Dense
import chainer
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from chainer.training import extensions
import os, sys, re
from nltk import tokenize
import nltk
nltk.download('punkt')
import string 

import pandas as pd

[nltk_data] Downloading package punkt to /home/bt978/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [2]:
import sys
print(sys.version)

2.7.6 (default, Mar 11 2014, 23:42:44) 
[GCC Intel(R) C++ gcc 4.4 mode]


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
TEXT_DATA_DIR = './speech_data/data/'

In [5]:
# Load the labels 
df = pd.read_csv('./speech_data/presidents_meta.csv')
label_dict = dict(zip(list(df.foldername), list(df.label)))

In [6]:
label_dict['arthur']

4

In [11]:
texts = []  # list of text samples
# labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids


for name in sorted(os.listdir(TEXT_DATA_DIR)):

    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        for fname in sorted(os.listdir(path)):
            if 'combined' not in fname and 'combines' not in fname:
                president_name = path.split('/')[-1]
                fpath = os.path.join(path, fname)
                f = open(fpath)
                if (president_name in label_dict):
                    labels.append(label_dict[president_name])
                    line = f.read()
                    line = re.sub(r'[^\x00-\x7F]+',' ', line).lower()
                    texts.append(line)
                f.close()
                

print('Found %s texts.' % len(texts))
print('Found %s texts.' % len(labels))

Found 862 texts.
Found 862 texts.


In [14]:
texts[500]

'the president. good afternoon, ladies and gentlemen.i have been asked to give a statement about the consular convention that is pending before the united states senate.i should like to say very briefly that i hope the senate will give its advice and consent to the proposed convention with the u.s.s.r. i feel very strongly that the ratification of this treaty is very much in our national interest. i feel this way for two principal reasons:first, we need this treaty to protect 18,000 american citizens who each year travel from this country to the soviet the convention requires immediate notification to us whenever an american is arrested in the soviet union. it insures our right to visit that citizen within 4 and as often thereafter as is desirable.we think that we need these rights help to protect american citizens. these are rights which the soviet citizens already have who travel in this country, because guaranteed by our constitution.second, the convention does not require the openi

In [17]:

    
def find_max_nb_words(texts):
    MAX_NB_WORDS = 0
    for text in texts:
        sents = tokenize.sent_tokenize(text)
        
        local_max = max(list(map(len, sents)))
        if   local_max > MAX_NB_WORDS  :
            MAX_NB_WORDS =  local_max
        #print("Maximum words {}".format(MAX_NB_WORDS))
    return MAX_NB_WORDS
MAX_NB_WORDS = find_max_nb_words(texts)

ValueError: max() arg is an empty sequence

In [19]:
MAX_NB_WORDS = 0

In [20]:
def normalize_corpus(text, replace_period=False):
    import string 
    sents = tokenize.sent_tokenize(text)
    sents = map(lambda x: x.replace('.',' '), sents)
    local_max = max(map(len, sents))
    if   local_max > MAX_NB_WORDS:
        MAX_NB_WORDS =  max(map(len, sents))
    print("Maximum words {}".format(MAX_NB_WORDS))
    text = ' . '.join(sents)      
    for char in string.punctuation:
        if not replace_period and char=='.':
            text = text.replace(char, ' . ') #text.replace(char, ' ' + char + ' ')
        else:
            text = text.replace(char, ' ') #text.replace(char, ' ' + char + ' ')
    text = re.sub(r'[^\x00-\x7F]+',' ', text) #remove non-ASCII chars
    text = re.sub( '\s+', ' ', text).lstrip().rstrip() #remove extra and trailing spaces
    return text.lower()
norm_texts = list(map(normalize_corpus, texts, MAX_NB_WORDS))

TypeError: argument 3 to map() must support iteration

In [ ]:
norm_texts[1]

In [ ]:
normalize_corpus(texts[500])

In [21]:
MAX_NB_WORDS = 100
len(texts)
mini_test = texts[:10]
print(list(map(len, mini_test)))
MAX_SEQUENCE_LENGTH = 100
MAX_SENTENCES = 100
import numpy as np

[2665, 11080, 10013, 1927, 81929, 62451, 51770, 54751, 5014, 6845]


In [22]:
def pad_dense_list(speech, maxsentence):
    
    each_sub_len = len(speech[0])
    Z = np.zeros((maxsentence, each_sub_len))
    #print("Shape of Z{}, list {}", Z.shape, len(speech))
    for index, row in enumerate(speech):
        if (index >= maxsentence):
            break
        
        Z[index] = row
    #print(Z) 
    return Z

In [23]:
def preprocess_text(mini_test):
    #mini_test = "mini test. hello world."
    tokenizer = Tokenizer(nb_words=5)
    tokenizer.fit_on_texts(mini_test)
    text_encoded = np.zeros((len(texts),MAX_SENTENCES,MAX_SEQUENCE_LENGTH))
    #print(text_encoded.shape)
    word_index = tokenizer.word_index
    for enu, speech in enumerate(mini_test):
        sents = tokenize.sent_tokenize(speech)
        if(len(sents) == 0):
            continue
        sequences = tokenizer.texts_to_sequences(sents)
        
        #print(len(sequences))
        #print('Found %s unique tokens.' % len(word_index))
        data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
        text_encoded[enu] = (pad_dense_list(data, 100))
    return word_index, text_encoded
word_index, text_encoded = preprocess_text(texts)


In [24]:
# First Speech encoding
data = text_encoded
labels = np.asarray(labels)
print(data.shape)

(862, 100, 100)


In [25]:
new_data = data

data = data.reshape(data.shape[0], data.shape[1]*data.shape[1])


In [49]:

data = np.zeros((800, 1000))
labels = np.zeros(800)

In [50]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
# labels = labels[indices]
VALIDATION_SPLIT = 0.2
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Shape of data tensor: (800, 1000)
Shape of label tensor: (800,)


In [51]:
embeddings_index = {}
GLOVE_DIR = '../glove.6B/'
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [52]:
EMBEDDING_DIM = 100

In [53]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [54]:
print(len(data))

800


In [55]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=data.shape[1],
                            trainable=False)

In [35]:
y_train = y_train.reshape((-1, 1))

In [47]:
print(len(x_train))

640


In [48]:
sequence_input = Input(shape=(data.shape[1],), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = LSTM(100)(embedded_sequences)
preds = Dense(7, activation='sigmoid')(x)
model = Model(sequence_input, preds)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=2, batch_size=10)


Train on 640 samples, validate on 160 samples
Epoch 1/2


KeyboardInterrupt: 

In [ ]:
# Experiment with sequence classification 
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words)

In [ ]:
(X_train_imdb, y_train_imdb), (X_test_imdb, y_test_imdb)  = (X_train, y_train), (X_test, y_test) 

In [ ]:
print(y_train_imdb[2])

In [ ]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
print(X_train[0])

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)